In [1]:
import json
import numpy as np
import getpass
import random
import pandas as pd 
seed = 42

# import test dataset 
# compute metrics
# save as a csv file

In [75]:
name = 'fake_dataset'
filename = f'{name}.json'

# file_name = 'test_dataset.json'
with open(filename, 'r') as file:
    test_dataset = json.load(file)

print("JSON file imported successfully.")

JSON file imported successfully.


In [58]:
def rag_recall(links, relevant_links):

    """
    Calculate the recall metric for a Retrieval-Augmented Generation (RAG) system.

    This function computes the recall by comparing the retrieved links to the relevant links.
    Recall is defined as the ratio of correctly retrieved relevant links to the total number
    of relevant links.

    Parameters:
    links (list): A list of retrieved links from the RAG system.
    relevant_links (list): A list of links that are considered relevant for the given query.

    Returns:
    float: The recall score, ranging from 0.0 to 1.0.
        - 1.0 if all relevant links are retrieved or if there are no relevant links.
        - 0.0 if no relevant links are retrieved.
        - Otherwise, the fraction of relevant links that were retrieved.

    Examples:
    >>> rag_recall(['1', '2', '3'], ['1', '2'])
    1.0
    >>> rag_recall(['1', '2', '3'], [])
    1.0
    >>> rag_recall(['1', '2', '3'], ['5'])
    0.0
    >>> rag_recall(['1', '2', '3'], ['1', '2', '3', '4'])
    0.75

    Note:
    - The function treats links as sets, so order and duplicates are ignored.
    - If more links are retrieved than are relevant, the maximum recall is still 1.0.
    """
    
    retrieved_links = set(links)
    relevant_set = set(relevant_links)
    
    num_correctly_retrieved = len(retrieved_links.intersection(relevant_set))
    num_relevant = len(relevant_set)

    if num_correctly_retrieved >= num_relevant:
        return 1.0
    elif num_relevant == 0:
        return 1.0
    elif num_correctly_retrieved == 0:
        return 0.0
    else:
        return num_correctly_retrieved/num_relevant



In [59]:
def get_recall_scores(test_dataset):
    
    recall_scores = [] 

    for i in range(len(test_dataset)):
        relevant_links = test_dataset[i]['links']
        links = test_dataset[i]['rag_links'] 
        score = rag_recall(links, relevant_links) 
        recall_scores.append({'question': test_dataset[i]['question'], 'recall': score })
    
    return recall_scores

recall_scores = get_recall_scores(test_dataset) 


In [81]:
pd.DataFrame(recall_scores)

,question,recall
0,"Welche Schritte muss ich unternehmen, um siche...",1.0
1,Welche spezifischen Kriterien müssen erfüllt s...,1.0
2,"Wie kann ich sicherstellen, dass meine Bewerbu...",1.0
3,"Wie kann ich sicherstellen, dass ich alle erfo...",1.0
4,"Wie kann ich herausfinden, welche Funktionen v...",1.0
5,Welche besonderen Veranstaltungen oder Themena...,1.0
6,"Warum muss ich den Continue-Button drücken, we...",0.0
7,Welche spezifischen Voraussetzungen müssen erf...,1.0
8,"Wie kann ich sicherstellen, dass meine persone...",1.0
9,Welche Informationen sind in der Wegleitung fü...,1.0


## Cosine Similarity 

In [5]:
openai_api_key = getpass.getpass("Enter your OpenAI API Key:\n\n")


Enter your OpenAI API Key:

 ········


In [6]:
# Checks connection to OpenAI 
import openai
from openai import OpenAI
client = OpenAI(api_key = openai_api_key)
try:
  #Make your OpenAI API request here
  response = client.chat.completions.create(
    messages=[{"role": "user", "content": "Hello world"}],
    model="gpt-4o-mini"
  )
except openai.APIError as e:
  #Handle API error here, e.g. retry or log
  print(f"OpenAI API returned an API Error: {e}")
  pass
except openai.APIConnectionError as e:
  #Handle connection error here
  print(f"Failed to connect to OpenAI API: {e}")
  pass
except openai.RateLimitError as e:
  #Handle rate limit error (we recommend using exponential backoff)
  print(f"OpenAI API request exceeded rate limit: {e}")
  pass

In [61]:
def get_embedding(text):
    """Generate an embedding for the given text using OpenAI's API."""

    # Check for valid input
    if not text or not isinstance(text, str):
        return None

    try:
        # Call OpenAI API to get the embedding
        openai.api_key = openai_api_key
        
        embedding = openai.embeddings.create(
            input=text,
            model="text-embedding-ada-002").data[0].embedding
        return embedding
    except Exception as e:
        print(f"Error in get_embedding: {e}")
        return None

In [62]:

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def compare_strings(str1: str, str2: str):
    # Get embeddings for both strings
    embedding1 = get_embedding(str1)
    embedding2 = get_embedding(str2)
    
    # Calculate cosine similarity
    similarity = cosine_similarity(embedding1, embedding2)
    
    return similarity



rec  = 0 
query = test_dataset[rec]['question']
human_answer =  test_dataset[rec]['answer']
rag_answer = test_dataset[rec]['rag_answer']
similarity_score = compare_strings(human_answer, rag_answer)

print('------------------------------------------------------------------------------')
print("Q: ", query)
print('------------------------------------------------------------------------------')
print("Human answer: ", human_answer)
print('------------------------------------------------------------------------------')
print("RAG answer:", rag_answer)

print('------------------------------------------------------------------------------') 
print(f"Semantic similarity between Human and RAG answers: {similarity_score:.4f}\n")


------------------------------------------------------------------------------
Q:  Welche Schritte muss ich unternehmen, um sicherzustellen, dass meine Bachelor- oder Masterarbeit den Anforderungen der Erklärung zur wissenschaftlichen Integrität entspricht, insbesondere in Bezug auf die Kennzeichnung von Quellen und die Verwendung von KI-unterstützter Technologie?
------------------------------------------------------------------------------
Human answer:  Um sicherzustellen, dass Ihre Bachelor- oder Masterarbeit den Anforderungen der Erklärung zur wissenschaftlichen Integrität entspricht, sollten Sie die folgenden Schritte unternehmen:

1. **Eigenständige Arbeit**: Stellen Sie sicher, dass Sie Ihre Arbeit unabhängig und ohne externe Hilfe verfasst haben. Dies bedeutet, dass Sie alle Ideen und Inhalte selbst entwickeln sollten.

2. **Quellenkennzeichnung**: Achten Sie darauf, alle verwendeten Quellen korrekt zu kennzeichnen. Dies umfasst sowohl direkte Zitate als auch Paraphrasierungen

In [63]:
similarity_scores = []

for rec in range(len(test_dataset)):
    query = test_dataset[rec]['question']
    human_answer =  test_dataset[rec]['answer']
    rag_answer = test_dataset[rec]['rag_answer']
    similarity_score = compare_strings(human_answer, rag_answer)
    similarity_scores.append({'question': query, 'cosine_similarity': similarity_score })

In [80]:
pd.DataFrame(similarity_scores)

,question,cosine_similarity
0,"Welche Schritte muss ich unternehmen, um siche...",0.989965
1,Welche spezifischen Kriterien müssen erfüllt s...,0.981745
2,"Wie kann ich sicherstellen, dass meine Bewerbu...",0.967187
3,"Wie kann ich sicherstellen, dass ich alle erfo...",0.963142
4,"Wie kann ich herausfinden, welche Funktionen v...",0.990407
5,Welche besonderen Veranstaltungen oder Themena...,0.989250
6,"Warum muss ich den Continue-Button drücken, we...",0.974779
7,Welche spezifischen Voraussetzungen müssen erf...,0.985502
8,"Wie kann ich sicherstellen, dass meine persone...",0.982969
9,Welche Informationen sind in der Wegleitung fü...,0.996162


## Jaccard Similarity

In [65]:
def jaccard_similarity(set1, set2):
    # Convert inputs to sets if they're not already
    set1 = set(set1)
    set2 = set(set2)
    
    # Calculate intersection and union
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    
    # Compute Jaccard Similarity
    similarity = len(intersection) / len(union)
    
    return similarity

def jaccard_similarity_text(text1, text2):
    # Convert texts to sets of words
    set1 = set(text1.lower().split())
    set2 = set(text2.lower().split())
    
    return jaccard_similarity(set1, set2)

# Example usage
text1 = "I like dogs"
text2 = "I hate dogs"

similarity = jaccard_similarity_text(text1, text2)
print(f"Jaccard Similarity: {similarity}")

Jaccard Similarity: 0.5


In [66]:
jaccard_scores = []

for rec in range(len(test_dataset)):
    query = test_dataset[rec]['question']
    human_answer =  test_dataset[rec]['answer']
    rag_answer = test_dataset[rec]['rag_answer']
    jaccard_similarity_score = jaccard_similarity_text(human_answer, rag_answer) 
    jaccard_scores.append({'question': query, 'jaccard_similarity': jaccard_similarity_score })

In [79]:
pd.DataFrame(jaccard_scores)

,question,jaccard_similarity
0,"Welche Schritte muss ich unternehmen, um siche...",0.524272
1,Welche spezifischen Kriterien müssen erfüllt s...,0.356863
2,"Wie kann ich sicherstellen, dass meine Bewerbu...",0.250000
3,"Wie kann ich sicherstellen, dass ich alle erfo...",0.240566
4,"Wie kann ich herausfinden, welche Funktionen v...",0.407609
5,Welche besonderen Veranstaltungen oder Themena...,0.574627
6,"Warum muss ich den Continue-Button drücken, we...",0.370690
7,Welche spezifischen Voraussetzungen müssen erf...,0.561224
8,"Wie kann ich sicherstellen, dass meine persone...",0.335052
9,Welche Informationen sind in der Wegleitung fü...,0.615385


## Unit Tests 


In [68]:

def generate_answer(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    response_content = response.choices[0].message.content
    return response_content
    
def unit_test_text(question, expected_answer, rag_answer):
    
    prompt_evaluation = """
    Question: {question}
    Expected Response: {expected_response}
    Actual Response: {actual_response}
    
    (Answer with 'True' or 'False') Does the Actual Response provide the necessary information for the question as the Expected Response does?
    """.strip()

    response = generate_answer(prompt_evaluation.format(question = question, expected_response = expected_answer, actual_response = rag_answer))

    return response
    
    

In [69]:
# rec = 0 
# query = test_dataset[rec]['question']
# human_answer =  test_dataset[rec]['answer']
# rag_answer = test_dataset[rec]['rag_answer']

# unit_test_text(query, human_answer, rag_answer)

'True'

In [70]:
rag_answer

'Um sicherzustellen, dass Ihre Bachelor- oder Masterarbeit den Anforderungen der Erklärung zur wissenschaftlichen Integrität entspricht, sollten Sie die folgenden Schritte unternehmen:\n\n1. **Selbstständige Erstellung der Arbeit**: Verfassen Sie Ihre Arbeit eigenständig und ohne fremde Hilfe. Dies ist eine grundlegende Voraussetzung.\n\n2. **Quellenkennzeichnung**: Achten Sie darauf, alle verwendeten Quellen korrekt zu kennzeichnen. Dies gilt sowohl für wörtliche Zitate als auch für sinngemäße Übernahmen. Stellen Sie sicher, dass Ihre Angaben über die benutzten Quellen wahrheitsgemäß und vollständig sind.\n\n3. **Kennzeichnung von KI-unterstützter Technologie**: Wenn Sie KI-gestützte Programme zur Erstellung von Textpassagen verwendet haben, müssen Sie diese Passagen entsprechend kennzeichnen. Geben Sie auch an, welches KI-unterstützte Programm Sie verwendet haben.\n\n4. **Plagiatsprüfung**: Seien Sie darauf vorbereitet, dass Ihre Arbeit auf Plagiate und die Verwendung von KI-unterstü

In [71]:
unit_tests = []

for rec in range(len(test_dataset)):
    query = test_dataset[rec]['question']
    human_answer =  test_dataset[rec]['answer']
    rag_answer = test_dataset[rec]['rag_answer']
    unit_tests.append({'question': query, 'unit_test': unit_test_text(query, human_answer, rag_answer) })



In [78]:
pd.DataFrame(unit_tests)

,question,unit_test
0,"Welche Schritte muss ich unternehmen, um siche...",True
1,Welche spezifischen Kriterien müssen erfüllt s...,True
2,"Wie kann ich sicherstellen, dass meine Bewerbu...",True
3,"Wie kann ich sicherstellen, dass ich alle erfo...",True
4,"Wie kann ich herausfinden, welche Funktionen v...",True
5,Welche besonderen Veranstaltungen oder Themena...,True
6,"Warum muss ich den Continue-Button drücken, we...",True
7,Welche spezifischen Voraussetzungen müssen erf...,True
8,"Wie kann ich sicherstellen, dass meine persone...",True
9,Welche Informationen sind in der Wegleitung fü...,True


## Save results in CSV 


In [110]:
# unit_tests
# jaccard_scores
# similarity_scores
# recall_scores

unit_test_df =  pd.DataFrame.from_dict(unit_tests)
jaccard_df =  pd.DataFrame.from_dict(jaccard_scores)
cosine_sim_df = pd.DataFrame.from_dict(similarity_scores)
recall_df = pd.DataFrame.from_dict(recall_scores)
df = pd.DataFrame.from_dict(test_dataset)


# merged_df = pd.DataFrame.from_dict(test_dataset).merge(
#     pd.DataFrame.from_dict(unit_tests), on='question', how='left'
# ).merge(
#     pd.DataFrame.from_dict(jaccard_scores), on='question', how='left'
# ).merge(
#     pd.DataFrame.from_dict(similarity_scores), on='question', how='left'
# ).merge(
#     pd.DataFrame.from_dict(recall_scores), on='question', how='left'
# )

#merged_df.to_csv(f'scored_{file_name}.csv', index=False, float_format='%.4f')


In [112]:
merged_df = pd.concat([df, unit_test_df['unit_test'],jaccard_df['jaccard_similarity'], cosine_sim_df['cosine_similarity'], recall_df['recall']], axis=1)
merged_df.to_csv(f'scored_{name}.csv', index=False, float_format='%.4f')

---
* Author: Anastasiia Popova
* Email: anastasiia.popova@stud.unibas.ch

[Perplexity AI](https://www.perplexity.ai/) assisted in code writing, editing, and more effective information searches. The generated output underwent critical evaluation. The author is solely responsible for the content.